In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

In [2]:
crawler = NewsCrawler()#'vice')

# 1. URL을 모은다

In [3]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [9]:
collect_results = crawler.collect(); display(*collect_results)

100.00% (321.30 seconds): Collecting URLs... theverge            

n_total     25593
n_unique    25524
dtype: int64

,pubs,actual_pub
https://bearingarms.com/tom-k/2019/10/08/connecticut-gun-theft-victim-gets-rest-of-his-guns-taken-by-police,"townhall, hotair",townhall
https://businessinsider.co.za/asylum-seekers-get-access-to-uif-with-labour-department-regulation-update-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/fnb-revamping-branches-in-bultfontein-daveyton-and-grassy-park-close-to-standard-bank-branches-that-have-shutdown-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/kfc-adding-chicken-wings-menu-colonel-rudy-sean-astin-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/saudi-arabia-laws-tourists-pitfalls-2019-10,"businessinsider, news24",businessinsider
https://hotair.com/archives/ed-morrissey/2019/10/09/media-honey-trap-doj-indicts-intel-expert-leaking-journalist-lover,"townhall, hotair",hotair
https://msnbc.com/morning-joe/watch/congressman-on-why-he-s-threatening-pompeo-s-salary-70923845715,"msnbc, nbcnews",msnbc
https://msnbc.com/morning-joe/watch/joe-obstructing-an-inquiry-likely-to-further-damage-trump-s-standing-70912069958,"msnbc, nbcnews",msnbc
https://msnbc.com/morning-joe/watch/trump-being-used-by-giuliani-says-conservative-writer-70913093972,"msnbc, nbcnews",msnbc
https://msnbc.com/morning-joe/watch/what-is-the-legal-remedy-for-trump-s-refusal-to-cooperate-70908997891,"msnbc, nbcnews",msnbc


huffpost                  68
cnn                      812
investing.com            967
politico                 214
time                      17
cnbc                     162
fox                      334
bbc                      279
businessinsider          543
morningstar               85
wsj                      107
nytimes                  103
guardian                 162
reuters                 1159
washingtontimes          477
washingtonpost           127
cbs                      403
marketwatch              151
atlantic                 101
vice                      99
npr                      869
newrepublic               20
yahoo                    409
independent              698
heritage                 286
zdnet                    388
townhall                 625
abcnews                   97
hotair                    84
cbc                      290
                        ... 
metro                    256
msnbc                    105
nationalreview           733
news24        

In [10]:
collect_results[2]['thinkprogress']

82

# 2. 모은 URL중 이미 받은 것은 버린다

In [11]:
select_results = crawler.select(); display(*select_results)

100.00% (62.83 seconds): Selecting URLs... cryptonews          

n_total     2276
n_unique    2276
dtype: int64

,pubs,actual_pub


huffpost                  3
cnn                      37
investing.com           153
politico                  9
time                      3
cnbc                      5
fox                      57
bbc                      19
businessinsider          97
morningstar              10
wsj                      24
nytimes                  10
guardian                 16
reuters                 236
washingtontimes          60
washingtonpost           40
cbs                      14
marketwatch              27
atlantic                  2
vice                     18
npr                       6
newrepublic               0
yahoo                   172
independent              22
heritage                  0
zdnet                    13
townhall                 18
abcnews                  16
hotair                   16
cbc                      25
                       ... 
metro                    35
msnbc                    14
nationalreview           32
news24                   36
techcrunch          

In [12]:
crawler.urls_selected['thinkprogress']

set()

# 3. 해당 URL의 뉴스를 다운로드한다

In [13]:
download_results = crawler.download(); download_results

48.59% (878.15 seconds): Downloading... axios               

exception calling callback for <Future at 0x18a4e6988d0 state=finished returned Article>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\concurrent\futures\_base.py", line 324, in _invoke_callbacks
    callback(self)
  File "C:\ProgramData\Anaconda3\lib\asyncio\futures.py", line 362, in _call_set_state
    dest_loop.call_soon_threadsafe(_set_state, destination, source)
  File "C:\ProgramData\Anaconda3\lib\asyncio\base_events.py", line 722, in call_soon_threadsafe
    self._check_closed()
  File "C:\ProgramData\Anaconda3\lib\asyncio\base_events.py", line 469, in _check_closed
    raise RuntimeError('Event loop is closed')
RuntimeError: Event loop is closed
exception calling callback for <Future at 0x18a40d27f28 state=finished returned Article>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\concurrent\futures\_base.py", line 324, in _invoke_callbacks
    callback(self)
  File "C:\ProgramData\Anaconda3\lib\asyncio\futures.py", line 362, in 

KeyboardInterrupt: 

In [20]:
import hashlib
[hashlib.sha1(url.encode('utf-8')).hexdigest() for url in crawler.urls_final['japan-news']['trashed']]

['d80494a4761ead4276e510dc4f64762e319902e0',
 'f25b1d06e58c05786ad57fe9bea9817859de3861',
 '02d668060004e25c2299c9a23445e38a9c6ec771',
 '2d3e330325b35d2c622806feac055478a4a61559',
 '2e87c621d1ffb7f28485496eb82814891ff5dd1b',
 '40cf0a5a2b87e040cd211da1855cb615702cb0ca',
 'feefa98b143ce76ef3cadf0a2764ca634f773fee',
 '0a900188f145b48aad8c70bde741aea563dba542',
 'ef7e33fa30c8be4b6d90681b383234ee4f73e51a',
 '64cc9a5e5a51520306154fda70d482b57a7420b0',
 '12e43a97340dc5a9f9270b38a9705e461456041b',
 'd71b4a23216b50deb276d3ccd8c1905939f5a9c2',
 '9a5e5f813637acd5359da986e562fa9ad9858f4b',
 '44c36bff1fe4d64fb468131f1221959fa30718d6',
 'dbf61f3dd462f9c650d0b2f42ca69717701fc7d1',
 '385dbc7ad54410ac2078055044e34c6e30cfde4b',
 'a31fe291585e194540805022e45df9ebdbabc9fc',
 '66afae226b2dc5d329fc731f7931f9f6d6f37e23']